In [18]:
import pandas as pd
import datetime as dt


base = pd.read_csv('datos/202106/2021-06_ KUNAN.xls - Base txt.csv', sep=",")

In [19]:
base.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10266 entries, 0 to 10265
Data columns (total 57 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Apellido y Nombre        10266 non-null  object 
 1   Tipo Documento           10266 non-null  object 
 2   Nro Documento            10266 non-null  int64  
 3   Estado Civil             0 non-null      float64
 4   Nacionalidad             0 non-null      float64
 5   Calle                    10266 non-null  object 
 6   Puerta                   10266 non-null  int64  
 7   Piso                     10266 non-null  int64  
 8   Dpto                     0 non-null      float64
 9   CP                       10266 non-null  int64  
 10  Telefono                 0 non-null      float64
 11  Fecha Nacimiento         10266 non-null  object 
 12  Sexo                     10266 non-null  object 
 13  CUIT                     0 non-null      float64
 14  CUIL                  

In [20]:
# Genero un dataframe con las columnas que necesito (agrege el numero de orden)
base_red = base[["Beneficio","Parentesco","CodDiagnostico","Tipo Diagnostico","Cod Prestación","Fecha Práctica", "Nro Orden", "Cantidad"]].copy()


# Cambio el tipo de dato fecha
base_red['Fecha Práctica'] = pd.to_datetime(base_red['Fecha Práctica'], dayfirst=True, format="%d/%m/%Y").dt.strftime('%d/%m/%Y')
    
#ordeno
base_red= base_red.sort_values(by=['Beneficio','Parentesco','CodDiagnostico','Fecha Práctica'])


In [21]:
base_red2 = base_red.copy()

In [22]:
#convierto a str 
base_red2 [['Beneficio', 'Parentesco']] = base_red2 [['Beneficio', 'Parentesco']].astype(str)

In [23]:
#agregar ceros
def agregar0(df, col: str, d: int):
    if (df[col].dtype == object):
        c = 0
        for i in df[col]:
            df.at[c,col] = i.zfill(d)
            c +=1
    return df[col]
    

In [24]:
base_red2 ['Beneficio'] =agregar0(base_red2, 'Beneficio', 12)
base_red2 ['Parentesco'] =agregar0(base_red2, 'Parentesco', 2)

In [25]:
base_red2

,Beneficio,Parentesco,CodDiagnostico,Tipo Diagnostico,Cod Prestación,Fecha Práctica,Nro Orden,Cantidad
9074,150671586901,00,G20,1,213001,01/06/2021,9918422326,30
9075,150671586901,00,G20,1,219101,01/06/2021,9918422326,1
9076,150671586901,00,G20,1,223101,01/06/2021,9918422326,1
9077,150671586901,00,G20,1,219101,02/06/2021,9918422326,1
9078,150671586901,00,G20,1,223101,02/06/2021,9918422326,1
...,...,...,...,...,...,...,...,...
110,020082001701,00,T12,1,223101,29/06/2021,9918754962,1
111,020082001701,00,T12,1,227012,29/06/2021,9918754962,1
112,020082001701,00,T12,1,219101,30/06/2021,9918754962,1
113,020082001701,00,T12,1,223101,30/06/2021,9918754962,1


In [26]:
from io import open

archivo = open("salida_ambulatorio.txt","w")

archivo.write("PRESTACIONES\n")

#variables
fecha = ""
matricula = "258089"
cuil = "30710160887"
vandera = False 

for index, row in base_red2.iterrows():
    if fecha != row['Fecha Práctica']:
        if (vandera):
            archivo.write("FIN AMBULATORIO\n")
            
        archivo.write("AMBULATORIO\n")
        archivo.write("{}{}{}{}{}{}{}{}{}{}{}{}".format(cuil, ";;", matricula, ";0;0;0;1;0;", row['Fecha Práctica'],";;;2;",row['Nro Orden'],";1;", row['Beneficio'],";",row['Parentesco'],"\n"))
        fecha = row['Fecha Práctica']
        diagnostico = ""
                      
        if diagnostico != row['CodDiagnostico']:
            archivo.write("REL_DIAGNOSTICOSXAMBULATORIO\n")
            archivo.write("{}{}{}{}{}{}{}".format(";;;0;1;", row['Tipo Diagnostico'],";", row['CodDiagnostico'],";",row['Tipo Diagnostico'], "\n"))
            archivo.write("REL_PRACTICASREALIZADASXAMBULATORIO\n")
            vandera = True 
                
    
                     
    #archivo.write("REL_PRACTICASREALIZADASXAMBULATORIO\n")
    archivo.write("{}{}{}{}{}{}{}{}{}".format(";;;0;1;", row['Cod Prestación'],";",fecha, ";", row['Cantidad'], ";0;",row['Nro Orden'],"\n"))
    




archivo.close()